<br><br><center><h1 style="font-size:3em">Praxisprojekt Datenbanken und SQL</h1></center>
<center><h3>Projekt 3 - ToDo Liste - Anton und Lisa</b></center>

### Standarteinstellungen

In [5]:
# Importe
import pandas as pd
import psycopg                        
import yaml                            

import sqlalchemy as sa 
import datetime as dt  
from sqlalchemy import text


In [6]:
# Standart-config-Datei laden
with open('..\\config.yaml', 'r') as file:
    config = yaml.load(file, Loader=yaml.BaseLoader)


### Neue Datenbank erstellen

In [18]:
# Connection-String
connection_string = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/" 

# Verbindungen aufbauen
sa_eng = sa.create_engine(connection_string, 
                          isolation_level='AUTOCOMMIT')

# Datenbank todo erstellen
with sa_eng.connect() as con:            
    con.execute(text('''CREATE DATABASE todo'''))

In [ ]:
# alte Verbindung trennen
sa_eng.dispose()

### Verbindung zur todo-Datenbank

In [9]:
# Verbindung zur todo-Datenbank erstellen
con_string_todo = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo"

# Verbindungen aufbauen
sa_eng = sa.create_engine(con_string_todo, 
                          isolation_level='AUTOCOMMIT')



### Tabellen erstellen und definieren

In [26]:
# TABELLE - PROJEKTE
# projekt_id serial PRIMARY KEY,
# projekt_name text NOT NULL,
# beschreibung text,
# startdatum date NOT NULL
# enddatum date NOT NULL

with sa_eng.connect() as con:            
    con.execute(text('''
                    CREATE TABLE projekte(
                        projekt_id SERIAL PRIMARY KEY,
                        projekt_name TEXT NOT NULL,
                        beschreibung TEXT,
                        startdatum DATE NOT NULL,
                        enddatum DATE);
                     '''))

# TABELLE - TEAM
# mitarbeiter_id PRIMARY KEY
# vorname
# nachname
# rolle (projektleitung, entwicklung, design,...)
# email

with sa_eng.connect() as con:            
    con.execute(text('''
        CREATE TABLE team(
            mitarbeiter_id SERIAL PRIMARY KEY,
            vorname TEXT NOT NULL,
            nachname TEXT NOT NULL,
            rolle TEXT,
            email TEXT NOT NULL
        );
    '''))

# TABELLE - AUFGABEN
# aufgaben_id int PRIMARY KEY
# aufgaben_name
# projekt_id int FOREIGN KEY
# aufgaben_beschreibung text
# mitarbeiter_id FOREIGN KEY
# status text
# erstellungsdatum
# faelligkeit

with sa_eng.connect() as con:            
    con.execute(text('''
        CREATE TABLE aufgaben(
            aufgaben_id SERIAL PRIMARY KEY,
            aufgaben_name TEXT NOT NULL,
            projekt_id INT,
            aufgaben_beschreibung TEXT,
            mitarbeiter_id INT,
            status TEXT,
            erstellungsdatum TIMESTAMP DEFAULT CURRENT_TIMESTAMP, 
            faelligkeit DATE,
            FOREIGN KEY (projekt_id) REFERENCES projekte(projekt_id),
            FOREIGN KEY (mitarbeiter_id) REFERENCES team(mitarbeiter_id)
        );
    '''))

### Klassen Definitionen

In [22]:
class Projekt:
    def __init__(self, projekt_id, projekt_name, beschreibung, startdatum, enddatum):
        self.projekt_id = projekt_id
        self.projekt_name = projekt_name
        self.beschreibung = beschreibung
        self.startdatum = startdatum
        self.enddatum = enddatum
    
        # Standart-config-Datei laden
        with open('..\\config.yaml', 'r') as file:
            config = yaml.load(file, Loader=yaml.BaseLoader)

        # Connection-String
        con_string_todo_list = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo"

        # Verbindung zur Datenbank
        self.sa_eng = sa.create_engine(con_string_todo_list, isolation_level='AUTOCOMMIT')

        # Metadaten und Auswahl Tabelle
        meta = sa.MetaData()
        self.projekte_table = sa.Table('projekte', meta, autoload_with=self.sa_eng)

    # Definition INSERT 
    def insert(self):
        # Einträge definieren
        ins = self.projekte_table.insert().values(
            projekt_name=self.projekt_name,
            beschreibung=self.beschreibung,
            startdatum=self.startdatum,
            enddatum=self.enddatum
        )
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(ins)
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Einfügen in die Datenbank: {e}")

    # Definition UPDATE 
    def update(self):
        # Einträge definieren
        upd = self.projekte_table.update().where(self.projekte_table.c.projekt_id == self.projekt_id).values(
                    projekt_name=self.projekt_name,
                    beschreibung=self.beschreibung,
                    startdatum=self.startdatum,
                    enddatum=self.enddatum
                    )
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(upd)
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Aktualisieren in der Datenbank: {e}")

    # Definition DELETE 
    def delete(self):
        # Einträge definieren
        delete_projekt = self.projekte_table.delete().where(self.projekte_table.c.projekt_id == self.projekt_id)
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(delete_projekt)
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Löschen aus der Datenbank: {e}")

In [25]:
class Team:
    def __init__(self, mitarbeiter_id, vorname, nachname, rolle, email):
        self.mitarbeiter_id = mitarbeiter_id
        self.vorname = vorname
        self.nachname = nachname
        self.rolle = rolle
        self.email = email
        
        # Standard-config-Datei laden und Verbindung zur Datenbank herstellen
        with open('..\\config.yaml', 'r') as file:
            config = yaml.safe_load(file)

        # Connection-String für SQLAlchemy
        con_string_todo_list = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo"

        # Verbindung zur Datenbank herstellen
        self.sa_eng = sa.create_engine(con_string_todo_list, isolation_level='AUTOCOMMIT')

        # Metadaten und Auswahl der Tabelle
        meta = sa.MetaData()
        self.team_table = sa.Table('team', meta, autoload_with=self.sa_eng)

    # Definition INSERT 
    def insert(self):
        # Einträge definieren und einfügen
        ins = self.team_table.insert().values(
            vorname=self.vorname,
            nachname=self.nachname,
            rolle=self.rolle,
            email=self.email
        )
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(ins)
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Einfügen in die Datenbank: {e}")

    # Definition UPDATE 
    def update(self):
        # Einträge definieren und aktualisieren
        upd = self.team_table.update().where(self.team_table.c.mitarbeiter_id == self.mitarbeiter_id).values(
            vorname=self.vorname,
            nachname=self.nachname,
            rolle=self.rolle,
            email=self.email
        )
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(upd)
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Aktualisieren in der Datenbank: {e}")

    # Definition DELETE 
    def delete(self):
        # Einträge definieren und löschen
        delete_teammitglied = self.team_table.delete().where(self.team_table.c.mitarbeiter_id == self.mitarbeiter_id)
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(delete_teammitglied)
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Löschen aus der Datenbank: {e}")

In [88]:
class Aufgaben:
    def __init__(self, aufgaben_id, aufgaben_name, 
                 aufgaben_beschreibung, 
                 projekt_id, mitarbeiter_id, status, 
                 erstellungsdatum, faelligkeit):
        self.aufgaben_id = aufgaben_id
        self.aufgaben_name = aufgaben_name
        self.aufgaben_beschreibung = aufgaben_beschreibung
        self.projekt_id = projekt_id
        self.mitarbeiter_id = mitarbeiter_id
        self.status = status
        self.erstellungsdatum = erstellungsdatum 
        self.faelligkeit = faelligkeit

        # Standart-config-Datei laden
        with open('..\\config.yaml', 'r') as file:
            config = yaml.load(file, Loader=yaml.BaseLoader)

        # Connection-String
        con_string_todo_list = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo"

        # Verbindung zur Datenbank
        self.sa_eng = sa.create_engine(con_string_todo_list, isolation_level='AUTOCOMMIT')

        # Metadaten und Auswahl Tabelle
        meta = sa.MetaData()
        self.aufgaben_table = sa.Table('aufgaben', meta, autoload_with=self.sa_eng)

    # Definition INSERT 
    def insert(self):
        # Einträge definieren
        ins = self.aufgaben_table.insert().values(
            aufgaben_name = self.aufgaben_name,
            aufgaben_beschreibung = self.aufgaben_beschreibung,
            projekt_id = self.projekt_id,
            mitarbeiter_id = self.mitarbeiter_id,
            status = self.status,
            faelligkeit = self.faelligkeit
        )
        
        with self.sa_eng.connect() as con:
            con.execute(ins)

            print(f"Die Aufgabe {self.aufgaben_name} wurde erfolgreich in die Tabelle eingefügt.")

            # Auswahl und Anzeige der Aufgaben mit der gleichen Projekt-ID
            query = self.aufgaben_table.select().where(
                    self.aufgaben_table.c.projekt_id == self.projekt_id)
            
            display(pd.read_sql(query, con))

    # Definition UPDATE 
    def update(self):
        # Einträge definieren
        upd = self.aufgaben_table.update().where(
            self.aufgaben_table.c.aufgaben_id == self.aufgaben_id).values(
            aufgaben_name=self.aufgaben_name,
            aufgaben_beschreibung=self.aufgaben_beschreibung,
            projekt_id=self.projekt_id,
            mitarbeiter_id=self.mitarbeiter_id,
            status=self.status,
            faelligkeit = self.faelligkeit
        )
        
        with self.sa_eng.connect() as con:
            con.execute(upd)

            print(f"Die Aufgabe {self.aufgaben_name} wurde erfolgreich in der Tabelle geändert.")

            # Auswahl und Anzeige der Aufgaben mit der gleichen Projekt-ID
            query = self.aufgaben_table.select().where(
                    self.aufgaben_table.c.projekt_id == self.projekt_id)
            
            display(pd.read_sql(query, con))

    # Definition DELETE 
    def delete(self):
        with self.sa_eng.connect() as con:
            # Aufgaben-ID abrufen, bevor gelöscht wird
            result = con.execute(text('SELECT projekt_id FROM aufgaben WHERE aufgaben_id = :id'), {'id': self.aufgaben_id}).fetchone()

            if result is None:
                print(f"Die Aufgabe mit ID {self.aufgaben_id} ist nicht vorhanden.")
                return

            self.projekt_id = result[0]

            # Aufgabe löschen
            con.execute(text('DELETE FROM aufgaben WHERE aufgaben_id = :id'), {'id': self.aufgaben_id})
            print(f"Die Aufgabe mit ID {self.aufgaben_id} wurde erfolgreich in der Tabelle gelöscht.")

            # Anzeige der verbleibenden Aufgaben mit der gleichen Projekt-ID
            if self.projekt_id is not None:
                query = con.execute(text('SELECT * FROM aufgaben WHERE projekt_id = :projekt_id'), {'projekt_id': self.projekt_id})
                df = pd.DataFrame(query.fetchall(), columns=query.keys())
                display(df)
            else:
                print(f"Fehler beim Löschen der Aufgabe: {self.aufgaben_id}")

In [91]:
# Erstellen einer Instanz mit der aufgaben_id 6
aufgabe = Aufgaben(16, None, None, None, None, None, None, None)  
aufgabe.delete()  

Die Aufgabe mit ID 16 wurde erfolgreich in der Tabelle gelöscht.


,aufgaben_id,aufgaben_name,projekt_id,aufgaben_beschreibung,mitarbeiter_id,status,erstellungsdatum,faelligkeit
0,17,ToDo-Liste Interface,3,Entwicklung eines Interfaces zur Bedienung der...,3,Offen,2024-07-16 09:51:01.963748,None


### Funktionen

In [ ]:
def info(mitarbeiter_id):
    query = text('''
        SELECT 
            t.vorname,
            t.nachname,
            p.projekt_name,
            a.aufgaben_name,
            a.status
        FROM 
            team t
        LEFT JOIN 
            aufgaben a ON t.mitarbeiter_id = a.mitarbeiter_id
        LEFT JOIN 
            projekte p ON a.projekt_id = p.projekt_id
        WHERE 
            t.mitarbeiter_id = :mitarbeiter_id;
    ''')
    
    with sa_eng.connect() as con:
        result = con.execute(query, {'mitarbeiter_id': mitarbeiter_id}).fetchall()
    
    if result:
        df = pd.DataFrame(result, columns=['Vorname', 'Nachname', 'Projekt Name', 'Aufgaben Name', 'Status'])
        return df
    else:
        return f"Keine Daten für Mitarbeiter-ID {mitarbeiter_id} gefunden."


In [ ]:
def get_all_aufgaben():
    query = text('''
        SELECT 
            a.aufgaben_id,
            a.aufgaben_name,
            p.projekt_name,
            t.vorname || ' ' || t.nachname AS mitarbeiter_name,
            a.aufgaben_beschreibung,
            a.status
        FROM 
            aufgaben a
        LEFT JOIN 
            projekte p ON a.projekt_id = p.projekt_id
        LEFT JOIN 
            team t ON a.mitarbeiter_id = t.mitarbeiter_id;
    ''')
    
    with sa_eng.connect() as con:
        result = con.execute(query).fetchall()
    
    if result:
        df = pd.DataFrame(result, columns=['Aufgaben ID', 'Aufgaben Name', 'Projekt Name', 'Mitarbeiter Name', 'Aufgaben Beschreibung', 'Status'])
        return df
    else:
        return "Keine Aufgaben gefunden."

In [ ]:
def set_status_erledigt(aufgaben_id):
    update_query = text('''
        UPDATE aufgaben
        SET status = 'Erledigt'
        WHERE aufgaben_id = :aufgaben_id;
    ''')
    
    with sa_eng.connect() as con:
        try:
            con.execute(update_query, {'aufgaben_id': aufgaben_id})
            print(f"Status der Aufgaben ID {aufgaben_id} wurde auf 'Erledigt' gesetzt.")
        except sa.exc.SQLAlchemyError as e:
            print(f"Fehler beim Aktualisieren des Status: {e}")


### Trigger und Log-Tabellen

In [34]:
# LOG Tabellen
with sa_eng.connect() as con:    
    # Log Tabelle Porjekte        
    con.execute(text('''
         CREATE TABLE projekte_log(
            log_projekt_id SERIAL PRIMARY KEY, 
            projekt_id INT NOT NULL,
            typ TEXT NOT NULL,
            projekt_name TEXT,
            aenderung TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
        )
    '''))

    # Log Tabelle Aufgaben
    con.execute(text('''
         CREATE TABLE aufgaben_log(
            log_aufgaben_id SERIAL PRIMARY KEY, 
            aufgaben_id INT NOT NULL,
            projekt_id INT NOT NULL,
            typ TEXT NOT NULL,
            aenderung TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
        )
    '''))
    
    # Log Tabelle Team
    con.execute(text('''
         CREATE TABLE team_log(
            log_team_id SERIAL PRIMARY KEY, 
            mitarbeiter_id INT NOT NULL,
            typ TEXT NOT NULL,
            aenderung TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
        )
    '''))

#### Trigger - Funktionen Projekt

In [35]:
# TRIGGER FUNKTIONEN projekt

with sa_eng.connect() as con:            
    # Trigger-Funktion für INSERT
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_projekte_insert() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO projekte_log (projekt_id, typ, projekt_name, aenderung)
            VALUES (NEW.projekt_id, 'INSERT', NEW.projekt_name, CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

    # Trigger-Funktion für UPDATE
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_projekte_update() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO projekte_log (projekt_id, typ, projekt_name,  aenderung)
            VALUES (OLD.projekt_id, 'UPDATE', OLD.projekt_name, CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

    # Trigger-Funktion für DELETE
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_projekte_delete() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO projekte_log (projekt_id, typ, projekt_name, aenderung)
            VALUES (OLD.projekt_id, 'DELETE', OLD.projekt_name, CURRENT_TIMESTAMP);
            RETURN OLD;
        END;
        $$ LANGUAGE plpgsql;
    ''')) 

In [36]:
# TRIGGER Tabelle Projekte

# Trigger für INSERT
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_projekte_insert_trigger
        AFTER INSERT ON projekte
        FOR EACH ROW
        EXECUTE FUNCTION log_projekte_insert();
    '''))

# Trigger für UPDATE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_projekte_update_trigger
        AFTER UPDATE ON projekte
        FOR EACH ROW
        WHEN (OLD.* IS DISTINCT FROM NEW.*)
        EXECUTE FUNCTION log_projekte_update();
    '''))

# Trigger für DELETE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_projekte_delete_trigger
        AFTER DELETE ON projekte
        FOR EACH ROW
        EXECUTE FUNCTION log_projekte_delete();
    '''))

#### Trigger - Funktionen Aufgaben

In [37]:
# TRIGGER FUNKTIONEN für Aufgaben

# Trigger-Funktion für INSERT
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_aufgaben_insert() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO aufgaben_log (aufgaben_id, projekt_id, typ, aenderung)
            VALUES (NEW.aufgaben_id, NEW.projekt_id, 'INSERT', CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

# Trigger-Funktion für UPDATE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_aufgaben_update() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO aufgaben_log (aufgaben_id, projekt_id, typ, aenderung)
            VALUES (OLD.aufgaben_id, OLD.projekt_id, 'UPDATE', CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

# Trigger-Funktion für DELETE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_aufgaben_delete() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO aufgaben_log (aufgaben_id, projekt_id, typ, aenderung)
            VALUES (OLD.aufgaben_id, OLD.projekt_id, 'DELETE', CURRENT_TIMESTAMP);
            RETURN OLD;
        END;
        $$ LANGUAGE plpgsql;
    '''))


In [38]:
# TRIGGER Tabelle Aufgaben

# Trigger für INSERT
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_aufgaben_insert_trigger
        AFTER INSERT ON aufgaben
        FOR EACH ROW
        EXECUTE FUNCTION log_aufgaben_insert();
    '''))

# Trigger für UPDATE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_aufgaben_update_trigger
        AFTER UPDATE ON aufgaben
        FOR EACH ROW
        WHEN (OLD.* IS DISTINCT FROM NEW.*)
        EXECUTE FUNCTION log_aufgaben_update();
    '''))

# Trigger für DELETE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_aufgaben_delete_trigger
        AFTER DELETE ON aufgaben
        FOR EACH ROW
        EXECUTE FUNCTION log_aufgaben_delete();
    '''))

#### Trigger - Funktionen Team

In [43]:
# TRIGGER FUNKTIONEN für Team

# Trigger-Funktion für INSERT
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_team_insert() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO team_log (mitarbeiter_id, typ, aenderung)
            VALUES (NEW.mitarbeiter_id, 'INSERT', CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

# Trigger-Funktion für UPDATE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_team_update() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO team_log (mitarbeiter_id, typ, aenderung)
            VALUES (OLD.mitarbeiter_id, 'UPDATE', CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

# Trigger-Funktion für DELETE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_team_delete() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO team_log (mitarbeiter_id, typ, aenderung)
            VALUES (OLD.mitarbeiter_id, 'DELETE', CURRENT_TIMESTAMP);
            RETURN OLD;
        END;
        $$ LANGUAGE plpgsql;
    '''))

In [44]:
# TRIGGER für Tabelle Team

# Trigger für INSERT
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_team_insert_trigger
        AFTER INSERT ON team
        FOR EACH ROW
        EXECUTE FUNCTION log_team_insert();
    '''))

# Trigger für UPDATE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_team_update_trigger
        AFTER UPDATE ON team
        FOR EACH ROW
        WHEN (OLD.* IS DISTINCT FROM NEW.*)
        EXECUTE FUNCTION log_team_update();
    '''))

# Trigger für DELETE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_team_delete_trigger
        AFTER DELETE ON team
        FOR EACH ROW
        EXECUTE FUNCTION log_team_delete();
    '''))

### Beispiel-Daten in Tabellen einfügen

In [46]:
# BEISPIELDATEN Tabelle PROJEKTE
projekt_1 = Projekt(None, 'Northwind Datenanalyse', 
            'Verbesserung der Datenverarbeitung und -auswertung für Northwind Traders mithilfe einer SQL-Datenbank.',
            '2024-01-01', '2024-02-15')
                   
projekt_2 = Projekt(None, 'Leihräder-Analyse', 
            'Umfassende Datenanalyse der Leihräder-Nutzung in San Francisco.',
            '2024-02-01', '2024-06-30')

projekt_3 = Projekt(None, 'ToDo-Liste', 
            'Entwicklung einer ToDo-Listen Anwendung in Python mit SQL-Datenbankverknüpfung.',
            '2024-03-01', '2024-05-31')
             
projekt_4 = Projekt(None, 'FIFA Fußball', 
            'Datenanalyse aller professionellen, europäischen Fußballspiele von 2008 bis 2016.',
            '2024-04-01', '2024-09-30')


# Insert Funktion anwenden
projekt_1.insert()
projekt_2.insert()
projekt_3.insert()
projekt_4.insert()

In [47]:
# BEISPIELDATEN Tabelle TEAM

team_1 = Team(None, 'Sarah', 'Müller', 'Projektleiterin', 'sarah.mueller@firma.de')

team_2 = Team(None, 'Johannes', 'Schmidt', 'Datenanalyst', 'johannes.schmidt@firma.de')

team_3 = Team(None, 'Derya', 'Toptas', 'Datenwissenschaftler', 'derya.toptas@firma.de')

team_4 = Team(None, 'Julian', 'Schneider', 'Dateningenieur', 'julian.schneider@firma.de')

team_5 = Team(None, 'Laura', 'Fischer', 'Business-Analystin', 'laura.fischer@firma.de')

# Insert Funktion anwenden
team_1.insert()
team_2.insert()
team_3.insert()
team_4.insert()
team_5.insert()

In [50]:
# BEISPIELDATEN Tabelle AUFGABEN
aufgabe_1 = Aufgaben(None, 'Northwind DB Aufbau', 
            'Erstellung und Verwaltung der SQL-Datenbank für Northwind.',
            1, 4, 'In Bearbeitung',None, None)

aufgabe_2 = Aufgaben(None, 'Northwind Analyse', 
            'Durchführung der Datenanalyse und Beantwortung der Fragen.',
            1, 2, 'In Bearbeitung',None, None)

aufgabe_3 = Aufgaben(None, 'Leihräder DB Aufbau', 
            'Import der Leihräder-Daten in die PostgreSQL-Datenbank und Einrichtung von Schlüsseln.',
            2, 4, 'In Bearbeitung',None, None)

aufgabe_4 = Aufgaben(None, 'Leihräder Analyse', 
            'Analyse der Leihräder-Daten und Erstellung von Berichten.',
            2, 2, 'In Bearbeitung',None, None)

aufgabe_5 = Aufgaben(None, 'ToDo-Liste Entwicklung', 
            'Entwicklung der ToDo-Listen Anwendung in Python und Verknüpfung mit SQL-Datenbank.',
            3, 3, 'In Bearbeitung',None, None)

aufgabe_6 = Aufgaben(None, 'ToDo-Liste Interface', 
            'Entwicklung eines Interfaces zur Bedienung der ToDo-Listen Anwendung.',
            3, 3, 'Offen',None, None)
            
aufgabe_7 = Aufgaben(None, 'FIFA DB Aufbau', 
            'Übertragung des FIFA-Datensatzes in eine neue PostgreSQL-Datenbank.',
            4, 4, 'Offen',None, None)

aufgabe_8 = Aufgaben(None, 'FIFA Analyse', 
            'Durchführung der Datenanalyse und Beantwortung der Fragen.',
            4, 2, 'Offen',None, None)

aufgabe_9 = Aufgaben(None, 'Ergebnispräsentation', 
            'Erstellung von Präsentationen zur Vorstellung der Analyseergebnisse.',
            4, 5, 'Offen',None, None)

# Insert Funktion anwenden
aufgabe_1.insert()
aufgabe_2.insert()
aufgabe_3.insert()
aufgabe_4.insert()
aufgabe_5.insert()
aufgabe_6.insert()
aufgabe_7.insert()
aufgabe_8.insert()
aufgabe_9.insert()

Die Aufgabe Northwind DB Aufbau wurde erfolgreich in die Tabelle eingefügt.


,aufgaben_id,aufgaben_name,projekt_id,aufgaben_beschreibung,mitarbeiter_id,status,erstellungsdatum,faelligkeit
0,3,Northwind DB Aufbau,1,Erstellung und Verwaltung der SQL-Datenbank fü...,4,In Bearbeitung,2024-07-16 09:31:20.352049,None
1,4,Northwind Analyse,1,Durchführung der Datenanalyse und Beantwortung...,2,In Bearbeitung,2024-07-16 09:31:20.371881,None
2,12,Northwind DB Aufbau,1,Erstellung und Verwaltung der SQL-Datenbank fü...,4,In Bearbeitung,2024-07-16 09:51:01.779312,None


Die Aufgabe Northwind Analyse wurde erfolgreich in die Tabelle eingefügt.


,aufgaben_id,aufgaben_name,projekt_id,aufgaben_beschreibung,mitarbeiter_id,status,erstellungsdatum,faelligkeit
0,3,Northwind DB Aufbau,1,Erstellung und Verwaltung der SQL-Datenbank fü...,4,In Bearbeitung,2024-07-16 09:31:20.352049,None
1,4,Northwind Analyse,1,Durchführung der Datenanalyse und Beantwortung...,2,In Bearbeitung,2024-07-16 09:31:20.371881,None
2,12,Northwind DB Aufbau,1,Erstellung und Verwaltung der SQL-Datenbank fü...,4,In Bearbeitung,2024-07-16 09:51:01.779312,None
3,13,Northwind Analyse,1,Durchführung der Datenanalyse und Beantwortung...,2,In Bearbeitung,2024-07-16 09:51:01.807219,None


Die Aufgabe Leihräder DB Aufbau wurde erfolgreich in die Tabelle eingefügt.


...

,aufgaben_id,aufgaben_name,projekt_id,aufgaben_beschreibung,mitarbeiter_id,status,erstellungsdatum,faelligkeit
0,18,FIFA DB Aufbau,4,Übertragung des FIFA-Datensatzes in eine neue ...,4,Offen,2024-07-16 09:51:02.004854,None


Die Aufgabe FIFA Analyse wurde erfolgreich in die Tabelle eingefügt.


,aufgaben_id,aufgaben_name,projekt_id,aufgaben_beschreibung,mitarbeiter_id,status,erstellungsdatum,faelligkeit
0,18,FIFA DB Aufbau,4,Übertragung des FIFA-Datensatzes in eine neue ...,4,Offen,2024-07-16 09:51:02.004854,None
1,19,FIFA Analyse,4,Durchführung der Datenanalyse und Beantwortung...,2,Offen,2024-07-16 09:51:02.050982,None


Die Aufgabe Ergebnispräsentation wurde erfolgreich in die Tabelle eingefügt.


,aufgaben_id,aufgaben_name,projekt_id,aufgaben_beschreibung,mitarbeiter_id,status,erstellungsdatum,faelligkeit
0,18,FIFA DB Aufbau,4,Übertragung des FIFA-Datensatzes in eine neue ...,4,Offen,2024-07-16 09:51:02.004854,None
1,19,FIFA Analyse,4,Durchführung der Datenanalyse und Beantwortung...,2,Offen,2024-07-16 09:51:02.050982,None
2,20,Ergebnispräsentation,4,Erstellung von Präsentationen zur Vorstellung ...,5,Offen,2024-07-16 09:51:02.099225,None


In [92]:
# Verbindung schließen
sa_eng.dispose()

### Tabellen anzeigen lassen

In [54]:
with sa_eng.connect() as con:
    aufgaben = pd.read_sql('aufgaben', con)

aufgaben

,aufgaben_id,aufgaben_name,projekt_id,aufgaben_beschreibung,mitarbeiter_id,status
0,1,Northwind DB Aufbau,1,Erstellung und Verwaltung der SQL-Datenbank fü...,4,In Bearbeitung
1,2,Northwind Analyse,1,Durchführung der Datenanalyse und Beantwortung...,2,In Bearbeitung
2,3,Leihräder DB Aufbau,2,Import der Leihräder-Daten in die PostgreSQL-D...,4,In Bearbeitung
3,4,Leihräder Analyse,2,Analyse der Leihräder-Daten und Erstellung von...,2,In Bearbeitung
4,5,ToDo-Liste Entwicklung,3,Entwicklung der ToDo-Listen Anwendung in Pytho...,3,In Bearbeitung
5,6,ToDo-Liste Interface,3,Entwicklung eines Interfaces zur Bedienung der...,3,Offen
6,7,FIFA DB Aufbau,4,Übertragung des FIFA-Datensatzes in eine neue ...,4,Offen
7,8,FIFA Analyse,4,Durchführung der Datenanalyse und Beantwortung...,2,Offen
8,9,Ergebnispräsentation,4,Erstellung von Präsentationen zur Vorstellung ...,5,Offen


In [55]:
with sa_eng.connect() as con:
    projekte = pd.read_sql('projekte', con)

projekte

,projekt_id,projekt_name,beschreibung,startdatum,enddatum
0,1,Northwind Datenanalyse,Verbesserung der Datenverarbeitung und -auswer...,2024-01-01,2024-02-15
1,2,Leihräder-Analyse,Umfassende Datenanalyse der Leihräder-Nutzung ...,2024-02-01,2024-06-30
2,3,ToDo-Liste,Entwicklung einer ToDo-Listen Anwendung in Pyt...,2024-03-01,2024-05-31
3,4,FIFA Fußball,"Datenanalyse aller professionellen, europäisch...",2024-04-01,2024-09-30


In [56]:
with sa_eng.connect() as con:
    team = pd.read_sql('team', con)

team

,mitarbeiter_id,vorname,nachname,rolle,email
0,1,Sarah,Müller,Projektleiterin,sarah.mueller@firma.de
1,2,Johannes,Schmidt,Datenanalyst,johannes.schmidt@firma.de
2,3,Derya,Toptas,Datenwissenschaftler,derya.toptas@firma.de
3,4,Julian,Schneider,Dateningenieur,julian.schneider@firma.de
4,5,Laura,Fischer,Business-Analystin,laura.fischer@firma.de


In [57]:
with sa_eng.connect() as con:
    team_log = pd.read_sql('team_log', con)

team_log

,log_team_id,mitarbeiter_id,typ,aenderung
0,1,1,DELETE,2024-07-15 14:04:05.155874
1,2,2,DELETE,2024-07-15 14:04:05.173670
2,3,3,DELETE,2024-07-15 14:04:05.175226
3,4,4,DELETE,2024-07-15 14:04:05.176740
4,5,5,DELETE,2024-07-15 14:04:05.178010
5,6,6,INSERT,2024-07-15 14:05:31.983899
6,7,7,INSERT,2024-07-15 14:05:31.993405
7,8,8,INSERT,2024-07-15 14:05:32.000074
8,9,9,INSERT,2024-07-15 14:05:32.005200
9,10,10,INSERT,2024-07-15 14:05:32.011260


In [58]:
with sa_eng.connect() as con:
    projekte_log = pd.read_sql('projekte_log', con)

projekte_log

,log_projekt_id,projekt_id,typ,projekt_name,aenderung
0,5,1,DELETE,Northwind Datenanalyse,2024-07-15 14:03:45.451141
1,6,2,DELETE,Northwind Datenanalyse,2024-07-15 14:03:45.456899
2,7,3,DELETE,Leihräder-Analyse,2024-07-15 14:03:45.457991
3,8,4,DELETE,ToDo-Liste,2024-07-15 14:03:45.458976
4,9,5,DELETE,FIFA Fußball,2024-07-15 14:03:45.460265
5,10,6,INSERT,Northwind Datenanalyse,2024-07-15 14:04:19.274508
6,11,7,INSERT,Leihräder-Analyse,2024-07-15 14:04:19.281950
7,12,8,INSERT,ToDo-Liste,2024-07-15 14:04:19.289406
8,13,9,INSERT,FIFA Fußball,2024-07-15 14:04:19.295972
9,14,6,UPDATE,Northwind Datenanalyse,2024-07-15 14:04:46.996194


In [59]:
with sa_eng.connect() as con:
    aufgaben_log = pd.read_sql('aufgaben_log', con)

aufgaben_log

,log_aufgaben_id,aufgaben_id,projekt_id,typ,aenderung
0,28,1,1,DELETE,2024-07-15 14:03:02.225252
1,29,2,1,DELETE,2024-07-15 14:03:02.232994
2,30,3,2,DELETE,2024-07-15 14:03:02.234190
3,31,4,2,DELETE,2024-07-15 14:03:02.235169
4,32,5,3,DELETE,2024-07-15 14:03:02.235900
...,...,...,...,...,...
65,93,13,3,DELETE,2024-07-15 14:18:01.786813
66,94,14,3,DELETE,2024-07-15 14:18:01.787516
67,95,15,4,DELETE,2024-07-15 14:18:01.788135
68,96,16,4,DELETE,2024-07-15 14:18:01.788751
